## [문제] Managing Conversation Hisory 모르겠다
- trim_messages()
- RunnablePassthroungh
- itemgetter()

세션 ID 설정하고, 대화를 진행합니다. (mult-turn conversation)
LLM 모델이 과거 대화를 알지(기억 = trimming) 못하는 상황을 만드세요

In [43]:
# 1. 모듈(파일, 라이브러리리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
# 2. 환경변수에 설정된 값 읽어오기
load_dotenv()
# 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]
prompt = ChatPromptTemplate.from_messages(template)
## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
# 6. chain 구성
chain = prompt | llm
# 7. 세션별 대화 이력(히스토리)를 저장할 임시 메모리 저장소
store = {}
## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
## 9. 사용자 질문 받기
query = input('육아휴직과 관련된 질문하세요 >> ')
## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)
## 11. 실행
with_message_history.invoke(
    {'law': law, 'query': query},
    config={'configurable': {'session_id': 'abc'}}
)

안녕하세요. 육아휴직 법률 전문가입니다. 질문 내용을 자세히 입력해 주시면, 문서 내용을 기반으로 성실하게 답변드리겠습니다. 혹시 궁금하신 사항이 있으시면 언제든 말씀해 주세요.

AIMessage(content='안녕하세요. 육아휴직 법률 전문가입니다. 질문 내용을 자세히 입력해 주시면, 문서 내용을 기반으로 성실하게 답변드리겠습니다. 혹시 궁금하신 사항이 있으시면 언제든 말씀해 주세요.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-33d82629-86ab-40a1-8632-390cf4c608fe-0')

In [41]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간 어떻게 돼?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직 기간은 「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」(이하 “남녀고용평등법”) 제19조에 따라 정해져 있습니다.\n\n- **기본 기간:** 육아휴직은 최대 1년 이내로 허용됩니다(제19조(육아휴직) ②항).\n- **추가 사용 조건:** 만약 자녀가 만 8세 이하 또는 초등학교 2학년 이하인 경우, 근로자가 이미 육아휴직을 3개월 이상 사용한 경우 그 자녀에 대해 다시 6개월 내에서 추가로 육아휴직을 사용할 수 있습니다(제19조(육아휴직) ②항).\n- **휴직 분할 사용:** 근로자는 육아휴직을 최대 3회에 나눠서 사용할 수 있으며, 임신 중인 여성 근로자가 모성보호를 위해 육아휴직을 사용한 경우는 이를 나누어 사용한 횟수에 포함하지 않습니다(제19조(육아휴직) ④항).\n\n추가적으로, 육아휴직 기간 동안은 근로자가 해고되지 않으며, 휴직 종료 후에는 이전과 동일하거나 같은 수준의 업무 또는 임금이 지급되는 직무에 복귀해야 합니다(제19조(육아휴직) ④항).\n\n혹시 더 구체적인 상황이나 조건에 따라 달라질 수 있으니, 상세한 내용은 근무하시는 사업장 또는 관련 기관에 문의하시는 것도 추천드립니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f'}, id='run-e5384199-6519-489a-9100-3e35679936f4-0')])}

In [ ]:
history